In [2]:
import ROOT
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import hist
import mplhep as hep

In [3]:
mpl.style.use(hep.style.ATLAS)
ROOT.gROOT.SetStyle('ATLAS')

# Propagazione degli errori in misure correlate e simulazione di esperimento: spettrometro a prisma
**M Sotgia<sup>1</sup>** \
_<sup>1</sup>Università degli studi di Genova, Dipartimento di Fisica_

## Introduzione

Un prisma ottico può essere utilizzato, sfruttando il fenomeno della rifrazione, come spettrometro (misuratore di lunghezza d'onda). 

Si sa che infatti la differenza $\delta_i$ tra l'angolo in ingresso $\theta_0$ e l'angolo in uscita $\theta_i$ risulta essere legato al valore dell'indice di rifrazione del materiale, $$\delta_i = \theta_0 - \alpha+\arcsin\left(n\sin\left(\alpha - \arcsin\left(\frac{\sin\theta_0}{n}\right)\right)\right),\tag{1}$$ con $n$ indice di rifrazione e $\alpha$ apertura angolare del prisma. 

Si osserva che $\delta_i$ ha un minimo in corrispondenza del quale la misura è più stabile e la relazione precedente si semplifica come $$n\sin\frac{\alpha}{2} = \sin\frac{\delta_m + \alpha}{2} = \sin\theta_{0_m}.\tag{2}$$

Possiamo anche però ricavare la relazione che lega l'indice di rifrazione $n$ al valore di $\delta_m$, e poichè $n=n(\lambda)$, secondo la relazione di Cauchy $$n(\lambda) = A + \frac{B}{\lambda^2},\tag{3} $$ appropriata ad un ordine $\mathcal O (1/\lambda^2)$, con $A$ e $B$ coefficienti propri del materiale in questione, allora possiamo concludere che esistono relazioni che legano $\lambda$ con il valore di $\delta_i$. 

## Misura sperimentale

Dal punto di vista sperimentale la misura di $\delta$ si effettua nel modo seguente

 1. si misura la direzione del raggio uscente in assenza di prisma ($\theta_0$), corrisponde ovviamente alla direzione del raggio incidente
 2. si ruota il prisma fino a trovare l'angolo di deviazione mimima  e misurare la direzione del raggio uscente ($\theta_i$)
 3. si calcola $\delta_i^m = \theta_i - \theta_0$ 

In [5]:
α = np.pi/3
λ_1 = 579.1 ## [nm] Y
λ_2 = 404.7 ## [nm] P
θ_0 = 1.5701
θ_1 = 2.6958
θ_2 = 2.7904

Δ_θ = 0.0005

Per trovare il valore di $A$ e $B$, noti i valori di $\theta_0$, $\theta_1$ e $\theta_2$, possiamo ottenere che questi sono legati dal sistema 

$$\left\{ \begin{align*}   &n_1 \sin\left( {\alpha \over 2} \right)
  = \sin\left( {{\theta_1-\theta_0+\alpha} \over 2} \right)\\
   &n_2 \sin\left( {\alpha \over 2} \right)
  = \sin\left( {{\theta_2-\theta_0+\alpha} \over 2} \right)\\
  &n_1 = A + {B\over \lambda^2_1}\\
  &n_2 = A + {B\over \lambda^2_2}\\
  \end{align*}
 \right.\tag{4}$$
 
Trovo quindi che i coefficienti $A$ e $B$ possono essere espressi rispetto ai parametri del problema nella forma $$ \left\{
\begin{align*}
n_i &= \frac{\sin\left( {{\theta_i-\theta_0+\alpha} \over 2} \right)}{\sin\left( {\alpha \over 2} \right)} && i = 1,2 \\
A &= \frac{\lambda_1^2 n_1 - \lambda_2^2 n_2}{\lambda_1^2 - \lambda_2^2} \\
B &= \frac{n_1 - n_2}{{1\over\lambda_1^2} - {1\over\lambda_2^2}} \\
\end{align*}
\right.$$

In [6]:
def computeAB(th_0, th_1, th_2):
    
    def n_i(th_i, th_0):
        return np.sin((th_i - th_0 + α)/2)/np.sin(α/2)
    A = (n_i(th_1, th_0)*λ_1**2 - n_i(th_2, th_0)*λ_2**2)/(λ_1**2 - λ_2**2)
    B = (n_i(th_1, th_0) - n_i(th_2, th_0))/(1/(λ_1**2) - 1/(λ_2**2))
    return A, B

## Metodo _variazionale_ per trovare A, B

Possiamo considerare la variazione dei valori di $\theta_i$ nell'errore, e quindi considerare una stima dell'errore di $A$ e $B$ in questo modo. 

In [ ]:
A, B = computeAB(th_0=θ_0, th_1=θ_1, th_2=θ_2)
